# Phase 2: Protocol Validation for Individual Subjects

This notebook contains results validating the AFID32 protocol on individual subjects from the OASIS-1 databank.

In [1]:
# initialize libraries
library(plyr)
library(digest)
library(reshape2)
library(ggplot2)
library("plot3D")

In [2]:
# useful functions

# calculate the distance between two sets of coordinates
dist3D <- function(coord1, coord2) { # vector X,Y,Z
        xdist <- coord1[1] - coord2[1] # could also write as coord1$X, etc.
        ydist <- coord1[2] - coord2[2]
        zdist <- coord1[3] - coord2[3]
        return(as.numeric(sqrt(xdist^2+ydist^2+zdist^2)))
}

# calculate the pairwise distance between an array of 3D coordinates
pairwise_dist3D <- function(temp_coords) { # labeled X,Y,Z
        N <- length(temp_coords$X)
        dist_vec <- rep(0,N) # create vector
        sum_dist <- 0 # initialize to zero
        count <- 0
        for (i in 1:(N-1)) {
                for (j in (i+1):N) {
                        if (i != j) {
                                count <- count + 1
                                first_coord <- temp_coords[i,]
                                second_coord <- temp_coords[j,]
                                curr_dist <- dist3D(first_coord, second_coord)
                                sum_dist <- sum_dist + curr_dist
                                dist_vec[count] <- curr_dist
                        }
                }
        }
        return(c(as.numeric(mean(dist_vec)),as.numeric(sd(dist_vec))))
}

# OAS1 Subset: Demographics

Demographics here.

In [3]:
df_OAS1 <- read.table('~/GitHub/afids-analysis/etc/OAS1_subjects.csv', sep=",", header=TRUE)
# include only MR1s
df_OAS1 <- subset(df_OAS1, session == 'MR1')

sprintf( "Total: %.1f +/- %.1f years; Range: %d-%d",
        mean(df_OAS1$age), sd(df_OAS1$age),
        min(df_OAS1$age), max(df_OAS1$age) )

sprintf( "Female: %d/%d (%.1f%%)",
        sum(df_OAS1$gender == 'F'), length(df_OAS1$gender),
        100*sum(df_OAS1$gender == 'F')/length(df_OAS1$gender) )

[1] "Total: 58.0 +/- 17.9 years; Range: 25-91"

[1] "Female: 17/30 (56.7%)"

In [4]:
# initialize variables and load in raw fcsv data into df_raters
setwd('~/GitHub/afids-analysis/data/PHASE2_input_afid/')

df_afid <- read.table('~/GitHub/afids-analysis/etc/afids.csv', sep=",", header=TRUE)

df_raters <- data.frame(uid=integer(), fid=integer(),X=double(),Y=double(),Z=double(),rater=factor(),
                        subject=factor(),mri_session=factor(),mri_type=factor(),session=integer(),date=integer(),
                        name=character(),description=character(),stringsAsFactors = FALSE)

csv_files <- list.files(".", "*.fcsv")

for (i in 1:length(csv_files)) {
        curr_split <- unlist(strsplit(csv_files[i],"_"))
        if (length(curr_split)>1) { # extract name and session data
                rater_subject <- paste(curr_split[1], curr_split[2],sep="_")
                rater_mri_session <- curr_split[3]
                rater_mri_type <- curr_split[4]
                rater_name <- curr_split[5]
                rater_session <- as.numeric(curr_split[6])
                rater_date <- as.numeric(unlist(strsplit(curr_split[7],"[.]"))[1])
                rater_filename <- paste(rater_subject,rater_mri_session,rater_mri_type,sep="_")
        }
        
        curr_rater <- read.table(csv_files[i], header=FALSE, sep=",")
        df_rater <- data.frame(fid = 1:length(curr_rater$V1))
        
        df_rater <- cbind(df_rater,X=curr_rater[2],Y=curr_rater[3],Z=curr_rater[4],rater=rater_name,
                          subject=rater_subject,mri_session=rater_mri_session,mri_type=rater_mri_type,
                          filename=rater_filename,session=rater_session,date=rater_date,name=curr_rater[12],
                          description=curr_rater[13])
        
        df_rater <- rename(df_rater, c("V2"="X","V3"="Y","V4"="Z","V12"="name","V13"="description"))
        df_raters <- rbind(df_raters,df_rater)
}

df_raters$uid <- seq.int(nrow(df_raters)) # add a unique identifier
levels(df_raters$rater) <- as.numeric( substr(levels(df_raters$rater), 6, 7 ) ) # rename raters based on rater number

# include only MR1s
df_raters <- subset(df_raters, mri_session == 'MR1')

In [5]:
# mean coordinates for each landmark placement for each subject/session
df_mean <- ddply(df_raters, .(subject, mri_session, fid, filename), summarize, X=mean(X), Y=mean(Y), Z=mean(Z))

# initialize
df_raters$mean_AFLE <- NA
df_raters$outlier <- NA

df_raters$xdist <- NA
df_raters$ydist <- NA
df_raters$zdist <- NA
for (i in 1:dim(df_raters)[1]) {
        curr_rater <- df_raters[i,]
        
        # set current mean based on subject, session, fid
        mean_raters <- subset(df_mean, subject == curr_rater$subject & mri_session == curr_rater$mri_session & fid == curr_rater$fid)
        
        df_raters[i,]$xdist <- curr_rater$X - mean_raters$X
        df_raters[i,]$ydist <- curr_rater$Y - mean_raters$Y
        df_raters[i,]$zdist <- curr_rater$Z - mean_raters$Z
        curr_coords <- curr_rater[,2:4]
        mean_coords <- mean_raters[,5:7]
        df_raters[i,]$mean_AFLE <- dist3D(curr_coords, mean_coords)
        df_raters[i,]$outlier <- (df_raters[i,]$mean_AFLE > 10) # focus on true outliers (1cm+) first
}

In [6]:
# summary of findings
all_subjects <- subset(df_raters, session > 0) # ignore session 0 which was from the group tutorial
num_outliers <- sum(subset(all_subjects, outlier == TRUE)$outlier)
num_total <- length(all_subjects$outlier)

sprintf( "Total: %.2f +/- %.2f mm; Outliers: %d/%d (%.2f%%)",
        mean(all_subjects$mean_AFLE), sd(all_subjects$mean_AFLE),
        num_outliers, num_total, (num_outliers/num_total)*100 )

# summary of the outliers
summary_outliers <- subset(df_raters,outlier==TRUE)[,c("fid","subject","mri_session","name","description","mean_AFLE")]
summary_outliers

# summary of results for each OASIS-1 scan that was annotated
summary_subjects_df <- ddply(df_raters, .(subject), summarize, mean=mean(mean_AFLE), sd=sd(mean_AFLE), max=max(mean_AFLE))
summary_afids_df <- ddply(df_raters, .(fid), summarize, mean=mean(mean_AFLE), sd=sd(mean_AFLE), max=max(mean_AFLE))

[1] "Total: 1.28 +/- 3.03 mm; Outliers: 28/2880 (0.97%)"

,fid,subject,mri_session,name,description,mean_AFLE
1373,29,OAS1_0203,MR1,29,R ventral occipital horn,16.19882
1501,29,OAS1_0216,MR1,29,R ventral occipital horn,17.77257
1502,30,OAS1_0216,MR1,30,L ventral occipital horn,11.61197
2043,27,OAS1_0256,MR1,28,L indusium griseum origin,15.80488
2044,28,OAS1_0256,MR1,27,R indusium griseum origin,15.35560
2141,29,OAS1_0263,MR1,29,R ventral occipital horn,39.35092
2142,30,OAS1_0263,MR1,30,L ventral occipital horn,40.64370
2173,29,OAS1_0263,MR1,29,R ventral occipital horn,78.74419
2174,30,OAS1_0263,MR1,30,L ventral occipital horn,80.42163
2205,29,OAS1_0263,MR1,29,R ventral occipital horn,39.39868


# Individual Subject Results: Post-QC

Re-analysis after quality control and filtering of outliers.

In [7]:
# now QC all the output
# some fiducials were initially mislabeled and corrected in postQC directory by consensus among raters

# initialize variables and load in raw fcsv data into df_raters
setwd('~/GitHub/afids-analysis/data/PHASE2_input_afid_postQC/')

df_afid <- read.table('~/GitHub/afids-analysis/etc/afids.csv', sep=",", header=TRUE)

df_raters <- data.frame(uid=integer(), fid=integer(),X=double(),Y=double(),Z=double(),rater=factor(),
                        subject=factor(),mri_session=factor(),mri_type=factor(),session=integer(),date=integer(),
                        name=character(),description=character(),stringsAsFactors = FALSE)

csv_files <- list.files(".", "*.fcsv")

for (i in 1:length(csv_files)) {
        curr_split <- unlist(strsplit(csv_files[i],"_"))
        if (length(curr_split)>1) { # extract name and session data
                rater_subject <- paste(curr_split[1], curr_split[2],sep="_")
                rater_mri_session <- curr_split[3]
                rater_mri_type <- curr_split[4]
                rater_name <- curr_split[5]
                rater_session <- as.numeric(curr_split[6])
                rater_date <- as.numeric(unlist(strsplit(curr_split[7],"[.]"))[1])
                rater_filename <- paste(rater_subject,rater_mri_session,rater_mri_type,sep="_")
        }
        
        curr_rater <- read.table(csv_files[i], header=FALSE, sep=",")
        df_rater <- data.frame(fid = 1:length(curr_rater$V1))
        
        df_rater <- cbind(df_rater,X=curr_rater[2],Y=curr_rater[3],Z=curr_rater[4],rater=rater_name,
                          subject=rater_subject,mri_session=rater_mri_session,mri_type=rater_mri_type,
                          filename=rater_filename,session=rater_session,date=rater_date,name=curr_rater[12],
                          description=curr_rater[13])
        
        df_rater <- rename(df_rater, c("V2"="X","V3"="Y","V4"="Z","V12"="name","V13"="description"))
        df_raters <- rbind(df_raters,df_rater)
}

df_raters$uid <- seq.int(nrow(df_raters)) # add a unique identifier
levels(df_raters$rater) <- as.numeric( substr(levels(df_raters$rater), 6, 7 ) ) # rename raters based on rater number

# mean coordinates for each landmark placement for each subject/session
df_mean <- ddply(df_raters, .(subject, mri_session, fid, filename), summarize, X=mean(X), Y=mean(Y), Z=mean(Z))

# initialize
df_raters$mean_AFLE <- NA
df_raters$outlier <- NA

df_raters$xdist <- NA
df_raters$ydist <- NA
df_raters$zdist <- NA
for (i in 1:dim(df_raters)[1]) {
        curr_rater <- df_raters[i,]
        
        # set current mean based on subject, session, fid
        mean_raters <- subset(df_mean, subject == curr_rater$subject & mri_session == curr_rater$mri_session & fid == curr_rater$fid)
        
        df_raters[i,]$xdist <- curr_rater$X - mean_raters$X
        df_raters[i,]$ydist <- curr_rater$Y - mean_raters$Y
        df_raters[i,]$zdist <- curr_rater$Z - mean_raters$Z
        curr_coords <- curr_rater[,2:4]
        mean_coords <- mean_raters[,5:7]
        df_raters[i,]$mean_AFLE <- dist3D(curr_coords, mean_coords)
        df_raters[i,]$outlier <- (df_raters[i,]$mean_AFLE > 10) # focus on true outliers (1cm+) first
}

df_raters_QC <- subset(df_raters, outlier == FALSE)

# mean coordinates for each landmark placement for each subject/session
df_mean_QC <- ddply(df_raters_QC, .(subject, mri_session, fid, filename), summarize, X=mean(X), Y=mean(Y), Z=mean(Z))

# initialize
df_raters_QC$mean_AFLE <- NA
df_raters_QC$outlier <- NA

df_raters_QC$xdist <- NA
df_raters_QC$ydist <- NA
df_raters_QC$zdist <- NA
for (i in 1:dim(df_raters_QC)[1]) {
        curr_rater <- df_raters_QC[i,]
        
        # set current mean based on subject, session, fid
        mean_raters <- subset(df_mean_QC, subject == curr_rater$subject & mri_session == curr_rater$mri_session & fid == curr_rater$fid)
        
        df_raters_QC[i,]$xdist <- curr_rater$X - mean_raters$X
        df_raters_QC[i,]$ydist <- curr_rater$Y - mean_raters$Y
        df_raters_QC[i,]$zdist <- curr_rater$Z - mean_raters$Z
        curr_coords <- curr_rater[,2:4]
        mean_coords <- mean_raters[,5:7]
        df_raters_QC[i,]$mean_AFLE <- dist3D(curr_coords, mean_coords)
        df_raters_QC[i,]$outlier <- (df_raters_QC[i,]$mean_AFLE > 10) # focus on true outliers (1cm+) first
}

# include only MR1s
df_raters_QC <- subset(df_raters_QC, mri_session == 'MR1')

In [8]:
# summary of findings
all_subjects <- subset(df_raters_QC, session > 0) # ignore session 0 which was from the group tutorial
num_outliers <- sum(subset(all_subjects, outlier == TRUE)$outlier)
num_total <- length(all_subjects$outlier)
sprintf( "Total: %.2f +/- %.2f mm; Outliers: %d/%d (%.2f%%)",
        mean(all_subjects$mean_AFLE), sd(all_subjects$mean_AFLE),
        num_outliers, num_total, (num_outliers/num_total)*100 )

# summary of results for each OASIS-1 scan that was annotated
summary_subjects_QC_df <- ddply(df_raters_QC, .(subject), summarize, mean=mean(mean_AFLE), sd=sd(mean_AFLE), max=max(mean_AFLE))
summary_afids_QC_df <- ddply(df_raters_QC, .(fid), summarize, mean=mean(mean_AFLE), sd=sd(mean_AFLE), max=max(mean_AFLE))


summary_AFLE_QC_df <- ddply(df_raters_QC, .(fid,subject), summarize, mean_AFLE=mean(mean_AFLE), sd_AFLE=sd(mean_AFLE), max_AFLE=max(mean_AFLE))
#summary_subjects_QC_df
#summary_afids_QC_df
#summary_AFLE_QC_df
#write.table(summary_AFLE_QC_df, file = "~/GitHub/afids-analysis/data/PHASE2_output_afid_postQC/PHASE2_subject_validation_AFLE.csv", row.names = FALSE, quote = FALSE, sep = ",")


[1] "Total: 0.94 +/- 0.73 mm; Outliers: 0/2872 (0.00%)"

In [9]:
####################################################################
# EXPORT MEAN FIDUCIAL LOCATIONS AS FCSV FILE (with outliers filtered out)
####################################################################
setwd('~/GitHub/afids-analysis/data/PHASE2_output_afid_postQC/')

for (curr_filename in levels(df_mean_QC$filename)) { # looping on each subject level
        curr_filename
        curr_mean <- subset(df_mean_QC, filename==curr_filename)
        curr_fcsv <- data.frame(id=paste('vtkMRMLMarkupsFiducialNode',curr_mean$fid,sep="_"),x=curr_mean$X,y=curr_mean$Y,z=curr_mean$Z,
                                       ow=0,ox=0,oy=0,oz=1,
                                       vis=1,sel=1,lock=1,label=curr_mean$fid,desc=df_afid$description,
                                       associatedNodeID='vtkMRMLScalarVolumeNode1',stringsAsFactors = FALSE)
        
        # write out table (need to use file connection approach because of header information)
        curr_fcsv_name <- paste0(curr_filename,'_MEAN.fcsv')
        fio <- file(curr_fcsv_name, open="wt")
        writeLines(paste('# Markups fiducial file version = 4.6'),fio)
        writeLines(paste('# CoordinateSystem = 0'),fio)
        writeLines(paste('# columns = id,x,y,z,ow,ox,oy,oz,vis,sel,lock,label,desc,associatedNodeID'),fio)
        write.table(curr_fcsv,fio,sep=',',quote=FALSE,col.names=FALSE,row.names=FALSE)
        close(fio)
}

# Inter-Rater AFLE

In [10]:
# inter-rater AFLE
#   defined here as the mean pairwise distance between mean intra-rater AFID coordinates

df_meanrater <- ddply(subset(df_raters_QC, session > 0 & outlier==FALSE & mri_session=="MR1"), .(subject,rater,fid), summarize, X=mean(X), Y=mean(Y), Z=mean(Z))

df_interrater <- data.frame(fid=integer(),
                            subject=factor(),
                            interrater_mean=double(),
                            interrater_sd=double(),
                            stringsAsFactors=FALSE)

for (curr_subject in levels(df_raters_QC$subject)) {
        for (curr_fid in 1:32) {
                curr_coords <- subset(df_meanrater, fid == curr_fid & subject == curr_subject)
                if (length(curr_coords$fid) > 0) {
                        curr_output <- pairwise_dist3D(curr_coords[,4:6]) # careful here as index can shift.
                        curr_df <- data.frame(fid = curr_fid, subject = curr_subject, interrater_mean = curr_output[1], interrater_sd = curr_output[2])
                        df_interrater <- rbind(df_interrater, curr_df)
                        
                }
        }
}

In [11]:
# exploration of inter-rater AFLE data
# summary of findings
summary_interrater_subjects_df <- ddply(df_interrater, .(subject), summarize, mean=mean(interrater_mean), sd=sd(interrater_mean), max=max(interrater_mean))

sprintf( "Total: %.2f +/- %.2f mm",
        mean(df_interrater$interrater_mean), sd(df_interrater$interrater_mean) )

summary_interrater_afids_df <- ddply(df_interrater, .(fid), summarize, mean=mean(interrater_mean), sd=sd(interrater_mean), max=max(interrater_mean))

[1] "Total: 1.58 +/- 1.02 mm"

In [12]:
# summary of findings
# combine all the fid metrics into one table; similarly across subjects pre- and post-QC
names(df_afid) <- c('fid','description','side')
summary_all_afids_df <- merge(df_afid, summary_afids_df, by = "fid")
summary_all_afids_df <- merge(summary_all_afids_df, summary_afids_QC_df, by = "fid")
summary_all_afids_df <- merge(summary_all_afids_df, summary_interrater_afids_df, by = "fid")
names(summary_all_afids_df) <- c('fid','description','side','mean_AFLE_mean','mean_AFLE_sd','mean_AFLE_max','mean_AFLE_mean_QC','mean_AFLE_sd_QC','mean_AFLE_max_QC','interrater_AFLE_mean_QC','interrater_AFLE_sd_QC','interrater_AFLE_max_QC')

summary_all_afids_df[,-3:-1] <- round(summary_all_afids_df[,-3:-1], 2)
#summary_all_afids_df

names(df_afid) <- c('fid','description','side')
summary_all_subjects_df <- merge(summary_subjects_df, summary_subjects_QC_df, by = "subject")
summary_all_subjects_df <- merge(summary_all_subjects_df, summary_interrater_subjects_df, by = "subject")
names(summary_all_subjects_df) <- c('subject','mean_AFLE_mean','mean_AFLE_sd','mean_AFLE_max','mean_AFLE_mean_QC','mean_AFLE_sd_QC','mean_AFLE_max_QC','interrater_AFLE_mean_QC','interrater_AFLE_sd_QC','interrater_AFLE_max_QC')

summary_all_subjects_df[,-3:-1] <- round(summary_all_subjects_df[,-3:-1], 2)
#summary_all_subjects_df

In [13]:
# combining both pre-QC and post-QC tables and formatting
df_afids <- read.table('~/GitHub/afids-analysis/etc/afids.csv', sep=",", header=TRUE)

combined_afids_pre_post_QC <- df_afids[,1:2]

combined_afids_pre_post_QC$mean_AFLE <- paste0( sprintf( "%.2f", round(summary_all_afids_df$mean_AFLE_mean,2)), '±', sprintf( "%.2f", round(summary_all_afids_df$mean_AFLE_sd,2)), ' (', sprintf( "%.2f", round(summary_all_afids_df$mean_AFLE_max,2)), ')')
combined_afids_pre_post_QC$mean_AFLE_postQC <- paste0( sprintf( "%.2f", round(summary_all_afids_df$mean_AFLE_mean_QC,2)), '±', sprintf( "%.2f", round(summary_all_afids_df$mean_AFLE_sd_QC,2)), ' (', sprintf( "%.2f", round(summary_all_afids_df$mean_AFLE_max_QC,2)), ')')
combined_afids_pre_post_QC$interrater_AFLE <- paste0( sprintf( "%.2f", round(summary_all_afids_df$interrater_AFLE_mean_QC,2)), '±', sprintf( "%.2f", round(summary_all_afids_df$interrater_AFLE_sd_QC,2)), ' (', sprintf( "%.2f", round(summary_all_afids_df$interrater_AFLE_max_QC,2)), ')')

names(combined_afids_pre_post_QC) <- c('AFID', 'Description', #'Side',
                                 'Mean AFLE Pre-QC', 'Mean AFLE Post-QC', 'Inter-Rater AFLE Post-QC'
)
combined_afids_pre_post_QC$AFID <- sprintf( "%02d", combined_afids_pre_post_QC$AFID )
combined_afids_pre_post_QC

#write.table(combined_afids_pre_post_QC, file = "~/GitHub/afids-analysis/data/output_tables/PHASE2_subject_validation_afid_AFLE_prepostQC.csv", row.names = FALSE, quote = FALSE, sep = ",")


AFID,Description,Mean AFLE Pre-QC,Mean AFLE Post-QC,Inter-Rater AFLE Post-QC
01,AC,0.36±0.21 (1.29),0.36±0.21 (1.29),0.60±0.25 (1.38)
02,PC,0.34±0.16 (0.88),0.34±0.16 (0.88),0.57±0.21 (1.22)
03,infracollicular sulcus,0.78±0.48 (3.07),0.78±0.48 (3.07),1.34±0.64 (3.84)
04,PMJ,0.83±0.49 (2.44),0.83±0.49 (2.44),1.41±0.55 (2.55)
05,superior interpeduncular fossa,1.20±0.75 (3.50),1.20±0.75 (3.50),2.04±0.90 (4.25)
06,R superior LMS,1.30±1.74 (14.25),1.01±0.55 (2.85),1.70±0.68 (3.13)
07,L superior LMS,1.36±1.71 (13.99),1.06±0.61 (3.45),1.72±0.71 (3.89)
08,R inferior LMS,1.13±0.75 (5.13),1.03±0.57 (2.99),1.77±0.74 (3.43)
09,L inferior LMS,1.10±0.80 (5.31),1.01±0.62 (2.72),1.71±0.86 (3.71)
10,culmen,0.99±0.99 (5.66),0.83±0.62 (3.07),1.35±0.82 (3.42)


In [14]:
combined_subjects_pre_post_QC <- df_OAS1[,1:2]

combined_subjects_pre_post_QC$mean_AFLE <- paste0( sprintf( "%.2f", round(summary_all_subjects_df$mean_AFLE_mean,2)), '±', sprintf( "%.2f", round(summary_all_subjects_df$mean_AFLE_sd,2)), ' (', sprintf( "%.2f", round(summary_all_subjects_df$mean_AFLE_max,2)), ')')
combined_subjects_pre_post_QC$mean_AFLE_postQC <- paste0( sprintf( "%.2f", round(summary_all_subjects_df$mean_AFLE_mean_QC,2)), '±', sprintf( "%.2f", round(summary_all_subjects_df$mean_AFLE_sd_QC,2)), ' (', sprintf( "%.2f", round(summary_all_subjects_df$mean_AFLE_max_QC,2)), ')')
combined_subjects_pre_post_QC$interrater_AFLE <- paste0( sprintf( "%.2f", round(summary_all_subjects_df$interrater_AFLE_mean_QC,2)), '±', sprintf( "%.2f", round(summary_all_subjects_df$interrater_AFLE_sd_QC,2)), ' (', sprintf( "%.2f", round(summary_all_subjects_df$interrater_AFLE_max_QC,2)), ')')

names(combined_subjects_pre_post_QC) <- c('Subject', 'Session', #'Side',
                                       'Mean AFLE Pre-QC', 'Mean AFLE Post-QC', 'Inter-Rater AFLE Post-QC'
)
combined_subjects_pre_post_QC <- combined_subjects_pre_post_QC[,c(1,3:5)]

#write.table(combined_subjects_pre_post_QC, file = "~/GitHub/afids-analysis/data/output_tables/PHASE2_subject_validation_subjects_AFLE_prepostQC.csv", row.names = FALSE, quote = FALSE, sep = ",")


# Secondary Analyses

We evaluated whether there was any evidence of an effect of demographics on AFLE.

In [15]:
# subset for age calculations (linear regression)
df_demographics <- merge(df_raters_QC, df_OAS1, by = 'subject')
l <- lm(mean_AFLE ~ age, data = df_demographics)
s <- summary(l) # for session, the estimate was positive: 0.0023
round(cbind(l$coeff,s$coefficients[,4]),4) # first column is the effect, second column is the pval

#cor.test(df_demographics$mean_AFLE, df_demographics$age, method = 'kendall')
#plot(df_demographics$age, df_demographics$mean_AFLE, xlim = c(20,100), ylim = c(0,10))


(Intercept),0.7694,0e+00
age,0.0030,1e-04


## Did AFLE worsen with the age of the subject for specific AFIDs?

We wanted to see if specific AFIDs tended to worsen with age of the OAS1 participant scan. Worsened for AFID17-18: bilateral LV at PC.

In [16]:
# Did raters improve placing specific AFIDs?
# create dataframe for linear model for each rater and p-values
models = dlply(df_demographics, .(fid), lm, formula = mean_AFLE ~ age)
# also extract p-values for intercept and session
qual <- laply(models, function(mod) summary(mod)$coefficients[,4])

coefs = ldply(models, coef)
summary_aging_afids <- cbind(coefs,qual)
summary_aging_afids <- summary_aging_afids[,c(1,2,4,3,5)]
names(summary_aging_afids)[c(3,5)] <- c('pval_(Intercept)','pval_session')

# FDR correction
summary_aging_afids$pval_session_adjusted <- p.adjust(summary_aging_afids$pval_session, "fdr")
summary_aging_afids$pval_session_significant <- (summary_aging_afids$pval_session_adjusted < 0.05)

# Round and display the table
summary_aging_afids[,c(2,4)] <- round( summary_aging_afids[,c(2,4)], 2)
summary_aging_afids[,c(3,5,6)] <- round( summary_aging_afids[,c(3,5,6)], 4)
summary_aging_afids

fid,(Intercept),pval_(Intercept),age,pval_session,pval_session_adjusted,pval_session_significant
1,0.19,0.0102,0.00,0.0133,0.1422,FALSE
2,0.24,0.0001,0.00,0.0964,0.3426,FALSE
3,0.93,0.0000,0.00,0.3885,0.5920,FALSE
4,0.86,0.0000,0.00,0.8868,0.9063,FALSE
5,0.81,0.0033,0.01,0.1364,0.4095,FALSE
6,1.24,0.0000,0.00,0.2292,0.4584,FALSE
7,0.66,0.0035,0.01,0.0572,0.2466,FALSE
8,0.79,0.0003,0.00,0.2276,0.4584,FALSE
9,0.60,0.0074,0.01,0.0557,0.2466,FALSE
10,0.61,0.0075,0.00,0.3133,0.5321,FALSE


In [17]:
sessionInfo()

R version 3.5.1 (2018-07-02)
Platform: x86_64-apple-darwin14.5.0 (64-bit)
Running under: macOS High Sierra 10.13.2

Matrix products: default
BLAS: /System/Library/Frameworks/Accelerate.framework/Versions/A/Frameworks/vecLib.framework/Versions/A/libBLAS.dylib
LAPACK: /System/Library/Frameworks/Accelerate.framework/Versions/A/Frameworks/vecLib.framework/Versions/A/libLAPACK.dylib

locale:
[1] en_CA.UTF-8/en_CA.UTF-8/en_CA.UTF-8/C/en_CA.UTF-8/en_CA.UTF-8

attached base packages:
[1] stats     graphics  grDevices utils     datasets  methods   base     

other attached packages:
[1] plot3D_1.1.1   ggplot2_3.0.0  reshape2_1.4.3 digest_0.6.16  plyr_1.8.4    

loaded via a namespace (and not attached):
 [1] Rcpp_0.12.17     compiler_3.5.1   pillar_1.3.0     bindr_0.1.1     
 [5] base64enc_0.1-3  tools_3.5.1      uuid_0.1-2       jsonlite_1.5    
 [9] evaluate_0.11    tibble_1.4.2     gtable_0.2.0     pkgconfig_2.0.2 
[13] rlang_0.2.1      IRdisplay_0.5.0  IRkernel_0.8.12  bindrcpp_0.2.2  
[17]